Importo le librerie necessarie per il progetto

In [1]:
import os
import csv
import cv2
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image,ImageOps
import matplotlib.pyplot as plt
import torchvision.transforms as T
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.models import resnet152
from torch.optim.lr_scheduler import ReduceLROnPlateau

Recupero il numero delle classi e verifico se la scheda video è impostata come device

In [2]:
train_path = r'C:\Users\alessio\Desktop\Project\AI\train'
test_path = r'C:\Users\alessio\Desktop\Project\AI\test'
image_path = r'C:\Users\alessio\Desktop\Project\AI\train\01\01_000.png'
image = Image.open(image_path)
num_classes = len(os.listdir(train_path))
print("n° Classi:", num_classes)
num_channels = len(image.getbands())
print("Numero di canali:", num_channels)
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

n° Classi: 8
Numero di canali: 3
True


Questa funzione prende in input un percorso relativo ad un'immagine (image_path), una bounding box (bbox) che specifica la regione di interesse dell'immagine e  l'immagine presa col path viene ritagliata in base ai valori della bounding box e normalizzata. 

In [3]:
def crop(image_path, bbox):
    # Estrae le coordinate del bounding box
    x_min, y_min, x_max, y_max = bbox

    # Carica l'immagine in formato BGR (Blu, Verde, Rosso)
    image_rgb = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Esegue il ritaglio dell'immagine utilizzando le coordinate del bounding box
    cropped_image = image_rgb[y_min:y_max, x_min:x_max]

    # Ridimensiona l'immagine ritagliata a una dimensione desiderata (280x280)
    resized_image = cv2.resize(cropped_image, (280, 280))

    # Normalizza i valori dei pixel nell'intervallo [0, 1]
    normalized_image = np.array(resized_image) / 255.0

    return normalized_image

Questo codice legge un file CSV contenente informazioni sulle immagini come:path, bounding box e etichette. Successivamente, manda ciascuna immagine in elaborazione alla funzione precedete, dopo memorizza l'immagine elaborata e l'etichetta associata in liste separate per l'addestramento di un modello di machine learning.

In [100]:
train_images = []  # Lista in cui verranno memorizzate le immagini elaborate
train_labels = []  # Lista in cui verranno memorizzate le etichette
train_df = pd.read_csv('train.csv')  # Carica i dati di addestramento da un file CSV

# Itera su ciascuna riga del DataFrame train_df
for i, (imp, x_min, y_min, x_max, y_max, label) in train_df.iterrows():
    # Costruisce il percorso completo dell'immagine da caricare
    image_path = os.path.join('AI/train', imp.replace('/', '\\'))
    
    # Assicura che le coordinate del bounding box siano non negative
    bbox = [x_min, y_min, x_max, y_max]
    for i in range(len(bbox)):
        if bbox[i] < 0:
            bbox[i] = 0

    # Chiama la funzione crop per ritagliare e preparare l'immagine
    image = crop(image_path, bbox)
    
    # Costruisce il nuovo percorso completo in cui salvare l'immagine
    image_path = os.path.join('AI/save', imp.replace('/', '\\'))
    
    # Ottiene le dimensioni dell'immagine risultante
    a, b, c = image.shape
    
    # Aggiunge l'etichetta e l'immagine alle liste di addestramento
    train_labels.append(label)
    train_images.append(image)

Questo codice converte le immagini e le etichette del dataset in tensori PyTorch, necessari per l'addestramento di modelli di deep learning. Inoltre, riorganizza le dimensioni delle immagini per adattarle al formato comunemente utilizzato da PyTorch.

In [101]:
train_images = np.array(train_images)  # Converte la lista di immagini in un array NumPy
train_labels = np.array(train_labels)  # Converte la lista di etichette in un array NumPy
labels = list(set(train_labels))  # Ottiene l'elenco unico delle etichette presenti nei dati di addestramento

# Stampa l'elenco delle etichette uniche
print("labels:", labels)

# Conversione delle immagini in tensori
train_images_tensor = torch.from_numpy(train_images).permute(0, 3, 1, 2).float()
train_labels_tensor = torch.from_numpy(train_labels).long()

labels: [0, 1, 2, 3, 4, 5, 6, 7]


Imposto la batch size e creo il dataset di addestramento, di validazione e i data loader.

In [103]:
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as T
from sklearn.model_selection import train_test_split

batch_size = 32  # Dimensione del batch per i data loader

# Divisione dei dati in set di addestramento (train) e di convalida (validation)
X_train, X_val, Y_train, Y_val = train_test_split(train_images_tensor, train_labels_tensor, test_size=0.2, random_state=42)

# Creazione dei dataset utilizzando TensorDataset
train_dataset = TensorDataset(X_train, Y_train)  # Dataset di addestramento
val_dataset = TensorDataset(X_val, Y_val)        # Dataset di convalida

# Crea i data loader per l'addestramento e la convalida
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

Questo modello si sulla trasformazione colorjitter, tale trasformazione aiuta il modello a generalizzare meglio poiche va a cambiare i colori all'interno dell'immagine inoltre viene fatto un crop a 224 per eliminare informazioni inutili dall'immagine e perche resnet lavora meglio con queste dimensioni.

In [123]:
# Caricamento del modello preallenato resnet152
from torchvision.models import resnet152
model = resnet152(weights="IMAGENET1K_V2")

# Definizione delle trasformazioni per il data augmentation
transf = T.ColorJitter(brightness=(0.2, 0.8), contrast=(0.2, 0.8), saturation=(1, 2), hue=(-0.5, 0.5))
transform = T.Compose([
    T.RandomApply(transf, 0.50),   # Applica trasformazioni con probabilità del 50%
    T.RandomHorizontalFlip(),     # Riflessione orizzontale casuale
    T.CenterCrop(224)             # Ritaglio centrale delle immagini a 224x224 pixel
])

# Modifica il classificatore finale del modello
n_inputs = model.fc.in_features  # Ottiene il numero di feature in input al classificatore
for param in model.parameters():
    param.requires_grad = True   # Scongela i pesi del modello

# Sostituisce il classificatore con uno personalizzato
model.fc = nn.Sequential(
    nn.Dropout(0.2),              # Dropout con probabilità del 20%
    nn.Linear(n_inputs, num_classes),  # Classificatore lineare con il numero di classi desiderato
    nn.LogSoftmax(dim=1)         # Log-softmax per la classificazione multiclasse
)

model = model.to(dev)  # Sposta il modello sulla GPU (o su un dispositivo specificato)

# Definizione della funzione di loss e dell'ottimizzatore
lr = 0.0001  # Tasso di apprendimento
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)  # Ottimizzatore Adam con regolarizzazione L2
criterion = nn.NLLLoss()  # Funzione di loss: log-likelihood negativa

# Definizione del learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.01, patience=5, verbose=True)

best_val_accuracy = 0.0
best_val_loss = 0.0
epoch = 0

# Loop di addestramento (può essere interrotto manualmente)
try:
    while True:
        # Fase di addestramento
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_images, batch_labels in train_loader:
            batch_images = transform(batch_images)  # Applica le trasformazioni di data augmentation
            batch_images = batch_images.to("cuda")  # Sposta le immagini sulla GPU
            batch_labels = batch_labels.to("cuda")  # Sposta le etichette sulla GPU
            optimizer.zero_grad()
            outputs = model(batch_images)  # Calcola le previsioni del modello
            loss = criterion(outputs, batch_labels)  # Calcola la loss
            loss.backward()  # Calcola i gradienti
            optimizer.step()  # Esegue un passo di ottimizzazione
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_labels.size(0)
            train_correct += (predicted == batch_labels).sum().item()
            train_loss += loss.item() * batch_images.size(0)

        train_accuracy = train_correct / train_total
        train_loss /= len(train_loader.dataset)

        # Fase di convalida
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch_images, batch_labels in val_loader:
                batch_images = transform(batch_images)
                batch_images = batch_images.to("cuda")
                batch_labels = batch_labels.to("cuda")
                outputs = model(batch_images)
                _, predicted = torch.max(outputs.data, 1)
                val_loss += criterion(outputs, batch_labels)
                val_total += batch_labels.size(0)
                val_correct += (predicted == batch_labels).sum().item()

        val_accuracy = val_correct / val_total
        val_loss /= len(val_loader)
        scheduler.step(val_accuracy)

        # Salvataggio dei pesi del modello
        torch.save(model.state_dict(), "model_4_" + str(epoch) + ".pth")

        print(f"Epoch [{epoch+1}] Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2%}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

        epoch += 1
        torch.cuda.empty_cache()

except KeyboardInterrupt:
    print("Addestramento interrotto manualmente")
    torch.save(model.state_dict(), "model_4_lastSave.pth")

Epoch [1] Training Loss: 1.0292, Training Accuracy: 78.44%, Validation Loss: 0.1884, Validation Accuracy: 96.88%
Epoch [2] Training Loss: 0.0607, Training Accuracy: 98.75%, Validation Loss: 0.0263, Validation Accuracy: 99.38%
Epoch [3] Training Loss: 0.0299, Training Accuracy: 99.45%, Validation Loss: 0.0892, Validation Accuracy: 97.19%
Epoch [4] Training Loss: 0.0113, Training Accuracy: 99.92%, Validation Loss: 0.0222, Validation Accuracy: 99.38%
Epoch [5] Training Loss: 0.0131, Training Accuracy: 99.61%, Validation Loss: 0.0148, Validation Accuracy: 99.69%
Epoch [6] Training Loss: 0.0079, Training Accuracy: 99.84%, Validation Loss: 0.0474, Validation Accuracy: 98.44%
Training interrotto manualmente


Questo codice legge da un file csv dei path relativi a delle immagini, per ogni immagine predice una label e scrive il risultato in un file csv.

In [13]:
# Definisci le trasformazioni per il data augmentation
data_transforms = T.Compose([
    T.ToTensor(),          # Converte l'immagine in un tensore
    T.CenterCrop(224)     # Esegue un ritaglio centrale delle immagini a 224x224 pixel
])
label_pre = []  # Lista per memorizzare le previsioni
model_path = 'model_4_4.pth'  # Percorso del modello allenato
model.load_state_dict(torch.load(model_path))  # Carica i pesi del modello allenato
model.to("cuda")  # Sposta il modello sulla GPU
model.eval()  # Imposta il modello in modalità di valutazione
test_df = pd.read_csv('test.csv')  # Carica il dataframe dei dati di test
output_file = 'submission.csv'  # Nome del file di output per le previsioni

# Apri il file di output per la scrittura
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image', 'class'])  # Scrivi l'intestazione del file CSV

    for i, (image_path, x_min, y_min, x_max, y_max) in test_df.iterrows():
        bbox = x_min, y_min, x_max, y_max  # Estrai le coordinate del bounding box
        image_ = os.path.join('AI/test', image_path.replace('/', '\\'))  # Costruisci il percorso dell'immagine
        image = crop(image_, bbox)  # Esegui il ritaglio dell'immagine
        image = data_transforms(image)  # Applica le trasformazioni
        image = image.float()  # Converti l'immagine in tipo float
        image = image.unsqueeze(0)  # Aggiungi una dimensione batch
        image = image.cuda()  # Sposta l'immagine sulla GPU

        with torch.no_grad():
            outputs = model(image)  # Calcola le previsioni del modello
            _, predicted = torch.max(outputs.data, 1)  # Trova la classe predetta
            predicted_label = predicted.item()  # Estrai la classe predetta come valore intero
            label_pre.append(predicted_label)  # Aggiungi la classe predetta alla lista
            writer.writerow([image_path, predicted_label])  # Scrivi l'immagine e la classe predetta nel file CSV

print("Il file CSV è stato generato con successo.")

Il file CSV è stato generato con successo.


Codice che testa le performance sulle immagini di test, creato per evitare upload inutili su kaagle.

In [14]:
# Precision test per evitare upload inutili
class_df = pd.read_csv('class.csv')  # Carica il dataframe contenente le etichette corrette
i = 0  # Inizializza un contatore
correct = 0  # Inizializza un contatore per le previsioni corrette

# Itera attraverso le etichette corrette e le previsioni
for label in class_df.iterrows():
    if int(label[1].values) == label_pre[i]:
        correct = correct + 1  # Incrementa il conteggio se la previsione è corretta
    i = i + 1  # Incrementa il conteggio generale delle etichette

# Calcola la percentuale di previsioni corrette
accuracy = correct / i

print("non corrette:", 800 - correct)  # Stampa il numero di previsioni incorrette
print(accuracy)  # Stampa l'accuratezza delle previsioni

non corrette: 2
0.9975
